# XGBoost

## 설명
- XGBoost란?: `extreme gradient boosting`를 의미하며, random forest와 마찬가지로 `ensemble` 방법론 중 하나다.
    - ensemble 방법론: 여러 모델의 predictions를 결합하는 기법(ex. random forest에서는 여러 tree의 결과를 결합)
    - 1개의 모델로 시작하며, 모델(들)의 loss를 줄이는 방향으로 모델을 추가해나간다.
- Cycle
    1. 현재 ensemble로 predictions를 구한다.
    2. predictions으로 `loss function`를 계산한다.
    3. loss function을 새로운 모델을 fit하는 데 사용한다. 새로 추가되는 모델은 loss를 줄여준다.
    4. 모델을 ensemble에 추가한다.
- Parameters
    - n_estimators: 몇 번의 사이클을 수행할 것인가?(사이클을 돌며 모델이 추가되기 때문에 모델 개수와도 일치)
        - 값이 너무 작으면 `underfitting`을 야기할 수 있음
        - 값이 너무 크면 `overfitting`을 야기할 수 있음
    - early_stopping_rounds: 연속으로 `validation score`가 개선되지 않으면 사이클을 멈출 것인가?
    - eval_set: early_stopping_rounds을 사용할 때 `validation score`를 계산하기 위한 데이터셋
    - learning_rate: 각 모델의 predictions에 곱해주는 값. 각 모델이 끼치는 영향력을 줄여준다(overfitting 방지).
- 이상적인 학습: `작은 learning_rate`과 `큰 n_estimators`로 학습할 때 일반적으로 가장 좋은 결과가 나온다.

## Exercise

### Flow
1. `XGBRegressor` 클래스를 통해 모델 정의
2. fit 모델
3. predict
4. MAE 계산

In [3]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [4]:
# Read the data
X = pd.read_csv('data/housing_prices_train.csv', index_col='Id')
X_test_full = pd.read_csv('data/housing_prices_test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [6]:
# Define the model
model = XGBRegressor(n_estimators=500,
                     learning_rate=0.05) # Your code here

# Fit the model
model.fit(X_train, y_train) # Your code here

# Get predictions
predictions = model.predict(X_valid) # Your code here

# Calculate MAE
mae = mean_absolute_error(y_valid, predictions) # Your code here

mae

16728.27523009418